In [1]:
import gspread
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import datetime

In [2]:
service_account = gspread.service_account(filename='./config/expense-tracker-358105-199d116b0a6d.json')
workbook = service_account.open('badminton_tracking')
worksheet = workbook.get_worksheet(0)

In [3]:
df = pd.DataFrame(worksheet.get_all_records()).drop(["Timestamp", "result"], axis=1)

df.columns = ["date", "team_1_player_1", "team_1_player_2", "team_2_player_1", "team_2_player_2", "points_team_1", "points_team_2", "venue"]

df['winner'] = np.where(df.points_team_1 > df.points_team_2, 'team_1', 'team_2')
df['margin'] = abs(df.points_team_1 - df.points_team_2)
df['date'] = pd.to_datetime(df['date']).dt.strftime("%Y-%m-%d")
df['total_points_per_game'] = df["points_team_1"] + df["points_team_2"]

df = df.applymap(lambda x: f'{x}'.lower().strip() if isinstance(x, str) else x)

### Settlements data

In [4]:
settlements_data = pd.DataFrame(workbook.worksheet('settlements').get_all_records()).set_index('date').reset_index()

settlements_data.groupby(['paid_by', 'paid_to']).sum('amount')

,,amount
paid_by,paid_to,
prateek,raghotham,368.33
srinidhi,raghotham,630.00
vinay,raghotham,1244.33


### Datewise expenditure

In [5]:
expense_data = pd.DataFrame(workbook.worksheet('expense_tracker').get_all_records()).set_index('date').reset_index()

In [6]:
expense_data.head()

,date,amount,paid_by
0,2023-03-05,380,ajay
1,2023-03-06,200,raghotham
2,2023-03-07,200,ajay
3,2023-03-08,270,raghotham
4,2023-01-03,500,raghotham


### players attended on date

In [9]:
def fn(x):
    return [i for i in set(', '.join([', '.join(x[i]) for i in range(4)]).split(', ')) if i != 'other']

players_on_date = df[['date', 'team_1_player_1', 'team_1_player_2', 'team_2_player_1', 'team_2_player_2']].copy()

players_on_date['date'] = players_on_date['date'].apply(lambda x: f'{datetime.strptime(x, "%Y-%m-%d").date()}')

players_on_date = players_on_date.groupby("date").agg({
    'team_1_player_1': 'unique',
    'team_1_player_2': 'unique',
    'team_2_player_1': 'unique',
    'team_2_player_2': 'unique'
})

players_on_date['players'] = players_on_date[['team_1_player_1', 'team_1_player_2', 'team_2_player_1', 'team_2_player_2']].apply(fn, axis=1)
players_on_date = players_on_date[['players']]
players_on_date['number_of_players'] = players_on_date['players'].str.len()

In [10]:
players_on_date.sort_values('date').tail()

,players,number_of_players
date,,
2023-03-27,"[raghotham, ajay, aakarsh, vinay]",4
2023-03-31,"[ajay, vinay, aakarsh, raghotham, srinidhi]",5
2023-04-01,"[ajay, vinay, aakarsh, raghotham, srinidhi]",5
2023-05-13,"[ajay, vinay, aakarsh, raghotham, sameer]",5
2023-05-14,"[ajay, vinay, raghotham, srinidhi, sameer]",5


### daily player share

In [11]:
player_share = pd.merge(players_on_date.explode('players'), expense_data, how="inner", on="date")
player_share['share'] = player_share['amount'] / player_share['number_of_players']

In [12]:
player_share.tail()

,date,players,number_of_players,amount,paid_by,share
176,2023-05-14,ajay,5,200,raghotham,40.0
177,2023-05-14,vinay,5,200,raghotham,40.0
178,2023-05-14,raghotham,5,200,raghotham,40.0
179,2023-05-14,srinidhi,5,200,raghotham,40.0
180,2023-05-14,sameer,5,200,raghotham,40.0


### total player balances

In [13]:
# balances = 
player_share.groupby('players').agg({
    'date': 'count',
    'share': 'sum'
}).round(decimals=2)

,date,share
players,,
aakarsh,23,1100.17
ajay,36,1740.33
nithin,5,213.33
prateek,12,584.33
raghotham,36,1740.33
ranga,1,50.00
sameer,18,775.00
srinidhi,17,896.17
vinay,33,1630.33


### Shuttle expenses

In [68]:
shuttle_expenses = pd.DataFrame(workbook.worksheet('shuttle_expense_tracker').get_all_records())

In [69]:
shuttle_expenses['covered_for'] = shuttle_expenses['shared_by'].str.split(', ')
shuttle_expenses['share'] = shuttle_expenses['amount'] / shuttle_expenses['covered_for'].apply(len)

shuttle_expenses = shuttle_expenses.explode('covered_for')[['expense_id', 'record_date', 'paid_by', 'amount', 'share', 'covered_for', 'comments']]
shuttle_expenses = shuttle_expenses[shuttle_expenses['paid_by'] != shuttle_expenses['covered_for']]

In [70]:
shuttle_expenses = shuttle_expenses[['paid_by', 'covered_for', 'expense_id', 'share']]
shuttle_expenses.columns = ['paid_by', 'players', 'date', 'share']

### player balances to others

In [71]:
balances = player_share.groupby(['players', 'paid_by'][::-1]).agg({
    'date': 'count',
    'share': 'sum'
}).reset_index()

In [76]:
pd.concat([balances, shuttle_expenses]).groupby(["paid_by", "players"]).agg({"share": "sum"}).reset_index()

,paid_by,players,share
0,aakarsh,ajay,875.000000
1,aakarsh,prateek,350.000000
2,ajay,aakarsh,200.000000
3,ajay,ajay,366.000000
4,ajay,nithin,50.000000
5,ajay,prateek,166.000000
6,ajay,raghotham,366.000000
7,ajay,sameer,90.000000
8,ajay,srinidhi,226.000000
9,ajay,vinay,316.000000


In [14]:


balances = balances[balances['paid_by'] != balances['players']]

In [15]:
balances = pd.merge(
    balances,
    balances,
    left_on=['paid_by', 'players'],
    right_on=['players', 'paid_by'],
    how="left"
)
# balances

In [16]:
balances['owes'] = np.where(
    balances['paid_by_y'].isna(),
    balances['share_x'],
    np.where(
        balances['share_x'] > balances['share_y'],
        balances['share_x'] - balances['share_y'],
        0
    )
)

In [17]:
balances = balances[['players_x', 'paid_by_x', 'owes', 'date_x']]

balances.columns = ['player', 'owes_to', 'amount', 'for_days']

In [18]:
balances

,player,owes_to,amount,for_days
0,aakarsh,ajay,200.000000,3
1,nithin,ajay,50.000000,1
2,prateek,ajay,166.000000,3
3,raghotham,ajay,0.000000,6
4,sameer,ajay,90.000000,2
5,srinidhi,ajay,226.000000,3
6,vinay,ajay,316.000000,5
7,aakarsh,raghotham,900.166667,20
8,ajay,raghotham,1008.333333,30
9,nithin,raghotham,163.333333,4


#### Post settlements

In [19]:
balances_post_settlement = pd.merge(
    balances,
    settlements_data,
    left_on=['player', 'owes_to'],
    right_on=['paid_by', 'paid_to'],
    how = "left"
)

balances_post_settlement['amount'] = np.where(
    balances_post_settlement['paid_to'].isna(),
    balances_post_settlement['amount_x'],
    np.where(
        balances_post_settlement['amount_x'] > balances_post_settlement['amount_y'],
        balances_post_settlement['amount_x'] - balances_post_settlement['amount_y'],
        0
    )
)

balances_post_settlement = balances_post_settlement[['player', 'owes_to', 'amount']].round(decimals=2)

In [20]:
balances_post_settlement

,player,owes_to,amount
0,aakarsh,ajay,200.00
1,nithin,ajay,50.00
2,prateek,ajay,166.00
3,raghotham,ajay,0.00
4,sameer,ajay,90.00
5,srinidhi,ajay,226.00
6,vinay,ajay,316.00
7,aakarsh,raghotham,900.17
8,ajay,raghotham,1008.33
9,nithin,raghotham,163.33


In [40]:
balances_post_settlement

,player,owes_to,amount
0,aakarsh,ajay,200.00
1,nithin,ajay,50.00
2,prateek,ajay,166.00
3,raghotham,ajay,0.00
4,sameer,ajay,90.00
5,srinidhi,ajay,226.00
6,vinay,ajay,316.00
7,aakarsh,raghotham,900.17
8,ajay,raghotham,1008.33
9,nithin,raghotham,163.33


In [52]:
shuttle_expenses_2 = shuttle_expenses[['covered_for', 'paid_by', 'share']].copy()
shuttle_expenses_2.columns = ['player', 'owes_to', 'amount']
shuttle_expenses_2['for_days'] = 1

In [57]:
pd.concat(
    [balances, shuttle_expenses_2]
).groupby(["player", "owes_to"]).agg({"amount": "sum"}).reset_index()

,player,owes_to,amount
0,aakarsh,ajay,200.000000
1,aakarsh,raghotham,1050.166667
2,ajay,aakarsh,875.000000
3,ajay,raghotham,1158.333333
4,nithin,ajay,50.000000
5,nithin,raghotham,163.333333
6,prateek,aakarsh,350.000000
7,prateek,ajay,166.000000
8,prateek,raghotham,568.333333
9,raghotham,ajay,0.000000


In [45]:
balances_post_settlement = balances_post_settlement.merge(
    shuttle_expenses[["covered_for", "paid_by", "share"]],
    left_on = ["player", "owes_to"],
    right_on = ["covered_for", "paid_by"],
    how="outer"
)

In [48]:
balances_post_settlement['amount'] = balances_post_settlement['amount'].fillna(0) + balances_post_settlement['share'].fillna(0)
balances_post_settlement['player'] = balances_post_settlement['player'].fillna(balances_post_settlement['covered_for'])
balances_post_settlement['owes_to'] = balances_post_settlement['owes_to'].fillna(balances_post_settlement['paid_by'])
balances_post_settlement

,player,owes_to,amount,covered_for,paid_by,share
0,aakarsh,ajay,200.00,NaN,NaN,NaN
1,nithin,ajay,50.00,NaN,NaN,NaN
2,prateek,ajay,166.00,NaN,NaN,NaN
3,raghotham,ajay,0.00,NaN,NaN,NaN
4,sameer,ajay,90.00,NaN,NaN,NaN
5,srinidhi,ajay,226.00,NaN,NaN,NaN
6,vinay,ajay,316.00,NaN,NaN,NaN
7,aakarsh,raghotham,1350.17,aakarsh,raghotham,150.0
8,ajay,raghotham,1458.33,ajay,raghotham,150.0
9,nithin,raghotham,163.33,NaN,NaN,NaN


### Venue wise expenditure

In [192]:
venue_wise_expenditure = pd.merge(
    expense_data,
    df[['date', 'venue']].drop_duplicates(),
    left_on='date',
    right_on='date'
).groupby('venue').sum('amount').reset_index()

In [194]:
venue_wise_expenditure

,venue,amount
0,infinity badminton academy,500
1,isro match point,950
2,match point - gublaala,3550
3,ssba - nps,500


In [282]:
px.pie(
    venue_wise_expenditure,
    values="amount",
    names='venue',
    template="plotly_white",
    color_discrete_sequence=px.colors.sequential.OrRd_r[-venue_wise_expenditure.shape[0]:],
    hole=0.3
)

### Monthy expenditure

In [239]:
monthly_expenditure = expense_data.copy()
monthly_expenditure['month'] = pd.to_datetime(monthly_expenditure['date']).dt.month

monthly_expenditure = monthly_expenditure.groupby(
    pd.to_datetime(monthly_expenditure['date']).dt.month_name()
).agg({'amount': 'sum', 'month': 'min'})

monthly_expenditure['color'] = np.where(
    monthly_expenditure['amount'] == monthly_expenditure['amount'].max(),
    '#b5de2b',
    'lightslategrey'
)

monthly_expenditure = monthly_expenditure.sort_values('month', ascending=False)

monthly_expenditure

,amount,month,color
date,,,
March,1250,3,lightslategrey
February,1900,2,lightslategrey
January,2350,1,#b5de2b


In [240]:
go.Figure(
    go.Bar(
        y=monthly_expenditure.index,
        x=monthly_expenditure['amount'],
        orientation='h',
        marker_color=monthly_expenditure['color'],
#         hovertemplate="Win Percentage: %{x} %"
    )
)